In [1]:
import time
import warnings
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
from tqdm import tqdm

In [2]:
import pandas as pd
df_og = pd.read_csv("data_index.csv")
factor_df = pd.read_pickle('factor_pkl.pkl')
data_bind = pd.read_pickle('data_bind.pkl')

In [4]:
data_bind

,종목코드,종목명,SEC_NM_KOR,ROE,GPA,CFO,DY,PBR,PCR,PER,PRS,12M,K_ratio,z_quality,z_value,z_momentum
0,000020,동화약품,건강관리,0.019096,0.403956,0.055600,0.0178,0.7508,10.8503,9.8295,0.7956,0.088362,4.591582,-2.851032,-5.060231,-1.195162
1,000040,KR모터스,경기관련소비재,-0.090473,0.091691,-0.025470,NaN,1.2765,NaN,NaN,0.5097,-0.044800,-12.228206,3.757001,NaN,0.333471
2,000050,경방,경기관련소비재,-0.004805,0.083892,0.022571,0.0127,0.3551,9.4324,NaN,0.7027,-0.284921,-61.718542,2.433971,NaN,3.066039
3,000070,삼양홀딩스,소재,0.008454,0.113031,0.033151,0.0493,0.2428,4.0565,7.1790,0.1848,0.062500,24.482365,0.431092,-5.677451,-1.500552
4,000080,하이트진로,필수소비재,0.015787,0.317866,-0.029284,0.0458,1.2817,NaN,20.2967,0.5781,-0.327817,-76.759593,-0.928571,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2393,456040,OCI,소재,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2394,456490,교보14호스팩,금융,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2395,457190,이수스페셜티케미컬,소재,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2396,460850,동국씨엠,소재,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = df_og[4000::]

In [4]:
df = df.dropna(axis = 1)

In [5]:
df.set_index('날짜', inplace=True)

In [6]:
df_per = df.pct_change()

In [7]:
sum(df_per.isna().sum())

112

In [8]:
class Environment1:
    """
    차트 데이터는 우선 간단하게 종가와 거래량만 제공한다.
    """
    """
    30일 데이터를 한꺼번에 넣은 다음 가중치를 출력한다. 
    """
    
    PRICE_IDX = 0  # 종가의 위치
    VOLUME_IDX = 1 # 거래량의 위치
    """
    version0: 아직 차트데이터에 가격밖에 없는 버전. 
    """
    
    
    def __init__(self, chart_data_df , feature_per_stock, factor_df = factor_df, data_bind = data_bind,balance=0, reward_method="return"):
        """
        chart data is n stocks that selected to portfolio
        """
        self.chart_data_df = chart_data_df                                    # type(chart_data) == pd.DataFrame
        self.feature_per_stock = feature_per_stock                      # feature per stock (i.e. close/volume)
        self.stock_num = int(chart_data_df.shape[1] / self.feature_per_stock)   # num of stock selected for portfolio
        self.idx = 0                                                    # index init
        self.state = chart_data_df.iloc[self.idx:self.idx+50]                          # state : Vector of the weight of the portfolio
#         self.next_state =   chart_data_df.iloc[self.idx+20:self.idx+40]                 # next state
        self.reward_method = reward_method                              # set method of reward   
        #self.init_vector = np.full(self.stock_num, 1/self.stock_num)    # set init_vector of weights 
        #self.factor_value = sum(self.state*self.init_vector)        # Portfolio growth compared to the previous day
        #self.factor_value_list = [self.portfolio_value]
        self.factor_df = factor_df
        self.data_bind = data_bind
    def reset(self):
        """
        initialize the all weights of portfolios equally
        """
        self.idx = 0                              # index init
        state = self.state
        return state
    
    def step(self, vector,data):
        """
        action is vertor that cons"i",st of weight of s-1ocks. 
        """
        if self.chart_data_df.shape[0] > self.idx + 50:
            reward,day_data = self.get_reward(vector,data)            
            self.idx += 4
            self.state = self.chart_data_df.iloc[self.idx:self.idx+50]
            #print(self.state.iloc[0].name, "~", self.state.iloc[-1].name)
            return self.state, reward, False, day_data

        else:
            self.idx = 0        
            reward,day_data = self.get_reward(vector,data)
            self.state = self.chart_data_df.iloc[self.idx:self.idx+50]
            
            #print(self.state.iloc[0].name, "~", self.state.iloc[-1].name)

            return self.state, reward, False,day_data

    
    def get_reward(self, vector,data):
        wts = vector ## factor 가중치
        data_bind_final_sum = (self.factor_df *wts).sum(axis=1,skipna=False).to_frame()
#         print("###")
        data_bind_final_sum.columns =['qvm']
        port_qvm = self.data_bind.merge(data_bind_final_sum, on='종목코드')
        port_qvm['invest'] = np.where(port_qvm['qvm'].rank() <= 20, 'Y','N')

        X = port_qvm[port_qvm['invest'] == 'Y']['종목코드']
        X = list(X)
        #print(X)
        reward,day_data = main2(data[X])  
        return reward, day_data
            


In [9]:
start = time.time()


# https://github.com/seungeunrho/minimalRL

#PPO-LSTM
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
import time
import numpy as np



class RECURRENT_PPO_1(nn.Module):
    def __init__(self):
        super(RECURRENT_PPO_1, self).__init__()
        self.data = []

        self.feature_num = 1629

        # 일단 32개의 주식을 포트폴리오로 구성한다고 가정 
        # when model action step, input of neural network is (stock_num * feture_per_stock) == [25 in version_0]
        # when model train step, input of neural network is (stock_num * feature_per_stock * T_horizon) [25 * 8 = 200 in version_0]

        self.fc1   = nn.Linear(64, 64)
        self.fc2   = nn.Linear(64,128)
        self.fc3   = nn.Linear(128,256)
        self.fc4   = nn.Linear(256, 128)

        self.lstm  = nn.LSTM(112, 64,2,batch_first=True)
        self.lstm_1 = nn.LSTM(112,64,2,batch_first=True)
        
        self.fc_pi_1 = nn.Linear(64,64)
        self.fc_pi_2   = nn.Linear(64,32)
        self.fc_pi_3 = nn.Linear(32, 3)

        self.fc_v_1 = nn.Linear(128,32)
        self.fc_v_2  = nn.Linear(32,1)
        learning_rate = 0.01
        momentum = 0.99  # Momentum parameter

# Create an instance of the SGD optimizer with momentum
        self.optimizer = optim.SGD(self.parameters(), lr=learning_rate, momentum=momentum)

    def pi(self, x, hidden,previous_vector):
        #out previous_vector
        #print(x.shape)
        if x.shape[0] == 50:
            x = x.unsqueeze(0)
        x, lstm_hidden = self.lstm(x.view(x.size(0), x.size(1), -1), hidden)
        x = self.fc_pi_1(x[:, -1, :])

        x = F.relu(self.fc_pi_1(x))
        x = F.relu(self.fc_pi_2(x))
        previous_vector = torch.tensor(previous_vector, dtype=torch.float32)
        
        x = F.relu(self.fc_pi_3(x))

        weight_vector = F.softmax(x, dim=1)
        return weight_vector, lstm_hidden
    
    def v(self, x, hidden):
        
        x, lstm_hidden = self.lstm_1(x, hidden)
        x = x[:, -1, :] 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))        

        v = self.fc_v_1(x)
        v = self.fc_v_2(v)
        
        return v

    def put_data(self, transition):
        self.data.append(transition)
        

    def make_batch(self):
        s_lst, a_lst, r_lst, s_prime_lst, h_in_lst, h_out_lst, done_lst, previous_vector_lst = [], [], [], [], [], [], [], []

        print(len(self.data))
        for transition in self.data:
            s, a, r, s_prime, h_in, h_out, done, previous_vector = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            h_in_lst.append(h_in)
            h_out_lst.append(h_out)
            done_mask = 0 if done else 1
            done_lst.append([done_mask])
            previous_vector_lst.append([previous_vector])
            
        s,a,r,s_prime,done_mask, previous_vector = torch.tensor(np.array(s_lst), dtype=torch.float), torch.tensor(a_lst), \
                                         torch.tensor(r_lst), torch.tensor(np.array(s_prime_lst), dtype=torch.float), \
                                         torch.tensor(done_lst, dtype=torch.float), torch.tensor(previous_vector_lst)
        
        self.data = []
        
#         print(len(s),len(a),len(r),len(s_prime), len(done_mask), len(h_in_lst[0]), len(h_out_lst[0]), len(previous_vector))
        return s,a,r,s_prime, done_mask, h_in_lst, h_out_lst, previous_vector
        #왜 h_in_lst[0]만 사용하지?
    

    def train_net(self):
        
        
        s, a, r, s_prime, done_mask, h_in_lst, h_out_lst, previous_vector = self.make_batch()

        # h_in_lst 내부의 텐서를 추출하고 적절하게 묶어서 first_hidden에 할당
        h_in_tensors = [inner_list[0] for inner_list in h_in_lst]  # 내부 리스트의 첫 번째 요소 추출
        first_hidden = (torch.cat([h.detach() for h in h_in_tensors], dim=1).detach(), torch.cat([h.detach() for h in h_in_tensors], dim=1).detach())

        # h_out_lst 내부의 텐서를 추출하고 적절하게 묶어서 second_hidden에 할당
        h_out_tensors = [inner_list[1] for inner_list in h_out_lst]  # 내부 리스트의 두 번째 요소 추출
        second_hidden = (torch.cat([h.detach() for h in h_out_tensors], dim=1), torch.cat([h.detach() for h in h_out_tensors], dim=1))

        # second_hidden 출력 # 2, 3, 64
        for i in range(K_epoch_1):
            a = self.v(s_prime,second_hidden)
            v_prime = a.squeeze(1)
            td_target = r + gamma_1 * v_prime
            v_s = self.v(s,first_hidden).squeeze(1)
            
            delta = td_target - v_s
            delta = delta.detach().numpy()

            advantage_lst = []
            advantage = 0.0
            for item in delta[::-1]:
                advantage = gamma_1 * lmbda_1 * advantage + item[0]
                advantage_lst.append([advantage])
            advantage_lst.reverse()
            advantage = torch.tensor(advantage_lst, dtype=torch.float)

            
#             print("###",s.shape, "###",first_hidden[0].shape, first_hidden[1].shape, "###",previous_vector.shape)
            vector, _ = self.pi(s.squeeze(0), first_hidden, previous_vector) # 애초에 h_in이 policy 신경망을 통과했기 때문에 first_hidden을 사용하는 것 같음.
            ratio = torch.exp(torch.log(vector) - torch.log(previous_vector))  # a/b == log(exp(a)-exp(b))

            surr1 = ratio * advantage
            surr2 = torch.clamp(ratio, 1-eps_clip_1, 1+eps_clip_1) * advantage
            loss = -torch.min(surr1, surr2) + F.smooth_l1_loss(v_s, td_target.detach())

            self.optimizer.zero_grad()
            loss.mean().backward(retain_graph=True)
            self.optimizer.step()

In [10]:
class Environment2:
    """
    차트 데이터는 우선 간단하게 종가와 거래량만 제공한다.
    """
    
    PRICE_IDX = 0  # 종가의 위치
    VOLUME_IDX = 1 # 거래량의 위치
    """
    version0: 아직 차트데이터에 가격밖에 없는 버전. 
    """
    
    
    def __init__(self, chart_data_df, feature_per_stock, balance=0, reward_method="return"):
        """
        chart data is n stocks that selected to portfolio
        """
        self.chart_data_df = chart_data_df                                    # type(chart_data) == pd.DataFrame
        self.feature_per_stock = feature_per_stock                      # feature per stock (i.e. close/volume)
        self.stock_num = int(chart_data_df.shape[1] / self.feature_per_stock)   # num of stock selected for portfolio
        self.idx = 0                                                    # index init
        self.state = chart_data_df.iloc[self.idx]                          # state : Vector of the weight of the portfolio
        self.next_state = chart_data_df.iloc[self.idx+1]                   # next state
        self.reward_method = reward_method                              # set method of reward   
        self.init_vector = np.full(self.stock_num, 1/self.stock_num)    # set init_vector of weights 
        self.portfolio_value = sum(self.state*self.init_vector)        # Portfolio growth compared to the previous day
        self.portfolio_value_list = [self.portfolio_value]
    
    def reset(self):
        """
        initialize the all weights of portfolios equally
        """
        self.idx = 0                                                    # index init
        state = self.state
        return state
    
    def step(self, vector):
        """
        action is vertor that consist of weight of stocks. 
        """
        try:
            self.state = self.chart_data_df.iloc[self.idx]       
            self.next_state = self.chart_data_df.iloc[self.idx+1]
            if self.chart_data_df.shape[0] > self.idx + 1:
                reward = self.get_reward(vector)
                next_state = self.next_state 
                done = False
                self.idx += 1
                name = self.state.name
                return next_state, reward, done, name
        except IndexError:
            return None, None, True, "reset"
    
    def get_reward(self, action):
        """
        reward
            - return: maximize profit of daytrading
            - sharpe: maximize sharpe ratio
        """
        if self.reward_method =="return":
            profit = sum(self.next_state * action) - self.portfolio_value_list[-1]   # in version 0, state has only close price
            reward = self.portfolio_value_list[-1] + profit
            self.portfolio_value_list.append(reward)
            return reward

        # not completed
        elif self.reward_method=="sharpe":
            reward = None
            return reward 

In [13]:
start = time.time()


# https://github.com/seungeunrho/minimalRL

#PPO-LSTM
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
import time
import numpy as np



class RECURRENT_PPO_2(nn.Module):
    def __init__(self, stock_num, all_feature_num):
        super(RECURRENT_PPO_2, self).__init__()
        self.data = []

        self.feature_num = all_feature_num
        self.stock_num = stock_num

        # 일단 32개의 주식을 포트폴리오로 구성한다고 가정 
        # when model action step, input of neural network is (stock_num * feture_per_stock) == [25 in version_0]
        # when model train step, input of neural network is (stock_num * feature_per_stock * T_horizon) [25 * 8 = 200 in version_0]
        input_shape = self.feature_num * self.stock_num
        self.lstm_2 = nn.LSTM(20,20,2)
        self.fc1   = nn.Linear(20, 64)
        self.fc2   = nn.Linear(64,128)
        self.fc3   = nn.Linear(128,256)
        self.fc4   = nn.Linear(256, 64)
        
        self.lstm_3  = nn.LSTM(128, 64,2)
        
        self.fc_pi_1 = nn.Linear(64,64)
        self.fc_pi_2   = nn.Linear(64,stock_num)
        self.fc_pi_3 = nn.Linear(2*stock_num, stock_num)

        self.fc_v_1 = nn.Linear(64,32)
        self.fc_v_2  = nn.Linear(32,1)
        
        learning_rate = 0.01
        momentum = 0.99  # Momentum parameter

# Create an instance of the SGD optimizer with momentum
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)

    def pi(self, x, hidden, previous_vector):
        y = self.fc1(x)
        nan_mask = torch.isnan(y)
        nan_values = y[nan_mask]
        if len(nan_values) != 0:
            print('x_-1',x)
        x = x.view(-1, 1, 20)
        x,lstm_hidden= self.lstm_2(x,hidden)
        
        x = self.fc1(x)
        nan_mask = torch.isnan(x)
        nan_values = x[nan_mask]
        if len(nan_values) != 0:
            print('x_--1',x)
        x = F.relu(x)
        nan_mask = torch.isnan(x)
        nan_values = x[nan_mask]
        if len(nan_values) != 0:
            print('x_0',x)
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))

       # x, lstm_hidden = self.lstm_3(x, hidden)

        nan_mask = torch.isnan(x)
        nan_values = x[nan_mask]
        if len(nan_values) != 0:
            print('x_1',x)
        x = F.relu(self.fc_pi_1(x))
        x = F.relu(self.fc_pi_2(x))
        
        previous_vector = torch.tensor(previous_vector, dtype=torch.float32)
        
        x = torch.concat([x, previous_vector.view(-1,1,self.stock_num)], 2)
        x = F.relu(self.fc_pi_3(x))
        nan_mask = torch.isnan(x)
        nan_values = x[nan_mask]
        if len(nan_values) != 0:
            print('x',x)
        weight_vector = F.softmax(x, dim=2)

        return weight_vector, lstm_hidden

    def v(self, x, hidden):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        

        

        v = self.fc_v_1(x)
        
        
        return v

    def put_data(self, transition):
        self.data.append(transition)

    def make_batch(self):
        s_lst, a_lst, r_lst, s_prime_lst, h_in_lst, h_out_lst, done_lst, previous_vector_lst = [], [], [], [], [], [], [], []


        for transition in self.data:
            s, a, r, s_prime, h_in, h_out, done, previous_vector = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            h_in_lst.append(h_in)
            h_out_lst.append(h_out)
            done_mask = 0 if done else 1
            done_lst.append([done_mask])
            previous_vector_lst.append([previous_vector])
            
        s,a,r,s_prime,done_mask, previous_vector = torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
                                         torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
                                         torch.tensor(done_lst, dtype=torch.float), torch.tensor(previous_vector_lst)
        
        self.data = []
        return s,a,r,s_prime, done_mask, h_in_lst[0], h_out_lst[0], previous_vector

    def train_net(self):
        s,a,r,s_prime,done_mask,(h1_in, h2_in), (h1_out, h2_out),previous_vector  = self.make_batch()
        first_hidden  = (h1_in.detach(), h2_in.detach())
        second_hidden = (h1_out.detach(), h2_out.detach())
        
        
        for i in range(K_epoch_2):
            v_prime = self.v(s_prime, second_hidden).squeeze(1)
            td_target = r + gamma_2 * v_prime * done_mask
            v_s = self.v(s, first_hidden).squeeze(1)
            delta = td_target - v_s
            delta = delta.detach().numpy()

            advantage_lst = []
            advantage = 0.0
            for item in delta[::-1]:
                advantage = gamma_2 * lmbda_2 * advantage + item[0]
                advantage_lst.append([advantage])
            advantage_lst.reverse()
            advantage = torch.tensor(advantage_lst, dtype=torch.float)

            

            vector, _ = self.pi(s, first_hidden, previous_vector) # 애초에 h_in이 policy 신경망을 통과했기 때문에 first_hidden을 사용하는 것 같음.
            ratio = torch.exp(torch.log(vector) - torch.log(previous_vector))  # a/b == log(exp(a)-exp(b))

            surr1 = ratio * advantage
            surr2 = torch.clamp(ratio, 1-eps_clip_2, 1+eps_clip_2) * advantage
            loss = -torch.min(surr1, surr2) + F.smooth_l1_loss(v_s, td_target.detach())

            self.optimizer.zero_grad()
            loss.mean().backward(retain_graph=True)
            self.optimizer.step()

In [14]:
#Hyperparameters
gamma_2        = 0.98
lmbda_2        = 0.95
eps_clip_2      = 0.1
K_epoch_2      = 4

count = 0
def main2(chart_data):
    stock_num = 20
    T_horizon  = 50
    env = Environment2(chart_data_df=chart_data, feature_per_stock=1)
    model = RECURRENT_PPO_2(stock_num=20, all_feature_num=20*1)
    score = 0.0
    epi_range = 10
    
    score_list = []
    print_interval = 12
    day_list = []
    score_list_1 = []
    day_list_1 = []

    for n_epi in range(epi_range):
        h_out = (torch.zeros([2, 1, 20], dtype=torch.float), torch.zeros([2, 1, 20], dtype=torch.float))
        done = False
        s = env.reset()
        previous_vector = np.full(stock_num, 1/stock_num)

        for t in range(T_horizon):
                #print('s',s)
                h_in = h_out
                vector, h_out = model.pi(torch.tensor(s, dtype=torch.float32), h_in, previous_vector)
                #print('vector',vector)
                vector = vector.view(-1)
                vector = vector.detach().numpy()

                s_prime, r, done, name = env.step(vector)
                #print("r,enpi:",r,n_epi)
                if not done:
                    model.put_data((s, vector, r, s_prime, h_in, h_out, done, previous_vector))
                    s = s_prime
                    previous_vector = vector
                    score += r
                    score_list_1.append([score,0])
                #print("주식별 가중치:\n", vector)
                
                if done:
                    s = env.reset()
                    break
        day_list.append(score_list_1)
        #print(score)
        score_list.append(score/70)
            
        score = 0.0
        model.train_net()
        #print("model1 score:",score)
    day_list_1.append(score_list_1)
    
    
    #print("model2 avg score:", sum(score_list)/epi_range)
    #print()
    #plt.plot(score_list)
    #plt.show()
    return sum(score_list)/epi_range, day_list_1



In [ ]:

#Hyperparameters
learning_rate_1 = 0.0005
gamma_1         = 0.98
lmbda_1         = 0.95
eps_clip_1      = 0.1
K_epoch_1       = 7
T_horizon_1     = 2
score_list_all_1 = []

def main1():
    env = Environment1(chart_data_df =df_per.loc['2023-01-30'::].iloc[:100], feature_per_stock=1)
    model = RECURRENT_PPO_1()
    score = 0.0
    score_list = []
    print_interval = 12
    day_data_0 = []
    day_data_1 = []
    day_data_2 = []
    
    for n_epi in tqdm(range(10)):
        h_out = (torch.zeros([2, 1, 64], dtype=torch.float), torch.zeros([2, 1, 64], dtype=torch.float))
        done = False
        s = env.reset()
        previous_vector = np.array([0.1,0.5,0.4])
        
        for t in range(T_horizon_1):
            h_in = h_out
            vector, h_out = model.pi(torch.tensor(s.values, dtype = torch.float), h_in,previous_vector)#out previous_vector
            vector = vector.view(-1)
            vector = vector.detach().numpy()
            s_prime, r, done, day_data = env.step(vector, s)

#           print("31번째 줄")
            model.put_data((s, vector, r, s_prime, h_in, h_out, done,previous_vector))#out previous_vector
            s = s_prime
            previous_vector = vector
            score += r
            score_list.append(r)
#            print("38번째 줄")
#                 break
#         print("43번째 줄")
            day_data_0.append(day_data)
        
        plt.plot(score_list)
        plt.show()
        model.train_net()
        day_data_1.append(day_data_0)
        print(sum(score_list)/T_horizon_1)
        print("episode:",n_epi)
        score_list_all.append(sum(score_list)/T_horizon_1)
        score = 0.0
        score_list = []
    day_data_2.append(day_data_1)
    return day_data_2
if __name__ == '__main__':
    a = main1()

end = time.time()

print(f"{end - start:.5f} sec")


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

In [ ]:
len(a[0][0][0][0])
#[의미없음][모델2의 Epoch][의미없음][0]

In [ ]:
plt.plot(score_list_all)

In [ ]:
score_list_all